# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import re
import numpy as np
from sqlalchemy import create_engine
from sklearn.datasets import make_multilabel_classification
from sklearn import multioutput
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

import nltk
nltk.download(['punkt', 'wordnet','stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import pickle
import warnings

import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)  
X = df['message']
Y = df.drop(columns=['id','message','original','genre'])
df.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products      ...        \
0        0      0            0             0                 0      ...         
1        0      0            1             0                 0      ...         
2        0      0            0             0                 0      ...         
3        1      0            1             0                 1      ...         
4        0      0            0             0                 0      ...         

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0            0                     0                0       0      0     0   
1            0                     0                1       0      1     0   
2            0                     0                0       0      0     0   
3            0                     0                0       0      0     0   
4            0                     0                0       0      0     0   

   earthquake  cold  other_weather  direct_report  
0           0     0              0              0  
1           0     0              0              0  
2           0     0              0              0  
3           0     0              0              0  
4           0     0              0              0  

[5 rows x 39 columns]

### 2. Write a tokenization function to process your text data

In [3]:
 """
 Tokenization function takes as input raw text which is normalized, stop words removed, 
 stemmed, and lemmatized, and the outcome is tokenized text returned.
 """
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    stop_words = stopwords.words("english")
    
    words = word_tokenize(text)
    
    lem = WordNetLemmatizer()
    
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    tokens = []
    
    tokens = [WordNetLemmatizer().lemmatize(w) for w in stemmed if w not in stop_words]
        
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', multioutput.MultiOutputClassifier (RandomForestClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.3, random_state = 999)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)

In [7]:
y_pred.shape

(7797, 35)

In [8]:
y_pred = pipeline.predict(X_test)
for i, col in enumerate(y_train):
    print('............................')
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

............................
related
             precision    recall  f1-score   support

          0       0.61      0.47      0.53      1797
          1       0.85      0.91      0.88      6000

avg / total       0.79      0.81      0.80      7797

............................
request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      6411
          1       0.80      0.45      0.58      1386

avg / total       0.87      0.88      0.87      7797

............................
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7771
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      7797

............................
aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.79      4529
          1       0.74      0.60      0.67      3268

avg / total       0.75      0.75      0.74  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters =  { 'clf__estimator__n_estimators': [20, 60,100], 
               'clf__estimator__max_depth': [4,8,12]} 

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=10)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.best_params_

In [ ]:
y_pred = cv.predict(X_test)
print(y_pred.shape)
for i, col in enumerate(y_train):
    print('............................')
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.ensemble import    AdaBoostClassifier
ada_forest = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)), ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(AdaBoostClassifier(base_estimator=RandomForestClassifier(),n_estimators=100, learning_rate=0.9,
                        random_state=None ))) ])

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    ada_forest.fit(X_train, y_train) 

In [ ]:
y_pred = ada_forest.predict(X_test)
print(y_pred.shape)
for i, col in enumerate(y_train):
    print('............................')
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

### 9. Export your model as a pickle file

In [ ]:
import pickle 
with open('classifier.pkl', 'wb') as file:
    pickle.dump(ada_forest, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.